# Basic EEG preprocessing

nRequire to install MNE and AutoReject packages

```
pip install mne
pip install autoreject
```

In [1]:
!pip install mne


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install autoreject

  Obtaining dependency information for autoreject from https://files.pythonhosted.org/packages/45/cf/f34035d2de261064090cb46491b1b999f0b9711bc57da189ffe1b50df146/autoreject-0.4.3-py3-none-any.whl.metadata
  Obtaining dependency information for scikit-learn>=0.24.2 from https://files.pythonhosted.org/packages/17/04/d5d556b6c88886c092cc989433b2bab62488e0f0dafe616a1d5c9cb0efb1/scikit_learn-1.6.1-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for h5io>=0.2.4 from https://files.pythonhosted.org/packages/20/8a/767a70e04e4532c05f2b60dc817672e5bfec4911ca6655e8943eadf80670/h5io-0.2.4-py3-none-any.whl.metadata
  Obtaining dependency information for pymatreader from https://files.pythonhosted.org/packages/bd/c7/56f50f023e19b0abd3ac93b86d64afdb58a16cca197441be9201bae367b7/pymatread


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Imports

In [1]:
import os
import logging
import mne
import numpy as np
import sys
from autoreject import AutoReject
from mne.utils import set_log_file

### Create loggers for preprocessing

In [2]:
######## PREPROCESSING ##############################################
# Create a custom logger for preprocessing INFO
logger_preprocessing_info = logging.getLogger('preprocessing_info')
logger_preprocessing_info.setLevel(logging.INFO)
logger_preprocessing_info.propagate = False

######## ERRORS ##############################################
# Create a custom logger for errors
logger_errors = logging.getLogger('errors')
logger_errors.setLevel(logging.INFO)

Bind loggers to file

NOTE: do it only one time

In [3]:
logger_dir_path = f'D:\studia\magisterka\dane EEG\BADANIE_POLITYCZNE_2022_eeg_bdfy\logs_from_preprocessing'

######## PREPROCESSING ##############################################
# Create a file handler for preprocessing and set the level to INFO
log_file_preprocessing = os.path.join(logger_dir_path, 'logger_preprocessing.txt')
file_handler_preprocessing = logging.FileHandler(log_file_preprocessing)
file_handler_preprocessing.setLevel(logging.INFO)

# Create a formatter and add it to the file handler for preprocessing
formatter_preprocessing = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler_preprocessing.setFormatter(formatter_preprocessing)

# Add the file handler for method A to the logger for preprocessing
logger_preprocessing_info.addHandler(file_handler_preprocessing)

######## ERRORS ##############################################
# Create a file handler for errors and set the level to INFO
log_file_error = os.path.join(logger_dir_path, 'logger_errors.txt')
file_handler_errors = logging.FileHandler(log_file_error)
file_handler_errors.setLevel(logging.INFO)

# Create a formatter and add it to the file handler for errors
formatter_errors = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler_errors.setFormatter(formatter_errors)

# Add the file handler for method A to the logger for preprocessing
logger_errors.addHandler(file_handler_errors)

##### MNE ###################################################
# Create logger for MNE logs
logger_f_name = f'./MNE-logs.txt'
set_log_file(fname=logger_f_name, output_format="%(asctime)s - %(message)s", overwrite=False)

### Preprocessing

In [4]:
def ocular_correction_gratton(epochs, subtract_evoked=False):

    if subtract_evoked:
        epochs_sub = epochs.copy().subtract_evoked()

        eog_model = mne.preprocessing.EOGRegression(
            picks="eeg",
            picks_artifact="eog"
        ).fit(epochs_sub)
    else:
        eog_model = mne.preprocessing.EOGRegression(
            picks="eeg",
            picks_artifact="eog"
        ).fit(epochs)

    epochs_clean_plain = eog_model.apply(epochs, copy=True)

    return epochs_clean_plain

In [6]:
def decimal_to_binary_string(decimal_number):
    decimal_number = int(decimal_number)
    binary_string = bin(decimal_number)[2:]
    binary_string_padded = binary_string.zfill(8)
    return binary_string_padded

def get_events(raw):
    data = raw.get_data('Status').flatten()
    ignored_values = [ 65280., 130816.,  65530.,  65535., 131071.]
    up_in, up_out, inv_in, inv_out = set(), set(), set(), set()
    for i in range(len(data)):
        x = data[i]
        if x not in ignored_values:
            y = x - 65280
            y = decimal_to_binary_string(y)
            mapping = {("0", "0"): up_in, ("1", "0"): up_out, ("0", "1"): inv_in, ("1", "1"): inv_out}
            mapping.get((y[0], y[1]), set()).add(x)

    events = mne.find_events(raw, stim_channel='Status', shortest_event= 1)
    new_events = mne.merge_events(events, up_in, 1)
    new_events = mne.merge_events(new_events, up_out, 2)
    new_events = mne.merge_events(new_events, inv_in, 3)
    new_events = mne.merge_events(new_events, inv_out, 4)
    return new_events

In [9]:
def preprocess_file(file_path, n_jobs=10, random_state=42):
  raw = mne.io.read_raw_bdf(
        file_path,
        eog=['EXG1', 'EXG2', 'EXG3', 'EXG4'], # should be channels with EOG signal, you might verify it
        exclude=['EXG5', 'EXG6', 'EXG7', 'EXG8'], # do not read channels we are not interested in
        preload=True
  )

  # 1. set montage
  try:
      raw = raw.set_montage('biosemi64')
  except ValueError as e:
      logger_errors.info('Lacks important channels!')

  # 2. re-reference: to average
  raw.set_eeg_reference(ref_channels='average')

  # 3. Notch filter
  power_freq = 50
  nyquist_freq = raw.info['sfreq'] / 2

  raw.notch_filter(
      picks=['eeg', 'eog'],
      freqs=np.arange(power_freq, nyquist_freq, power_freq),
      n_jobs=n_jobs,
  )

  # 4. Filters epochs
  raw_filtered = raw.copy().filter(
      picks=['eeg', 'eog'],
      l_freq=.05,
      h_freq=30.0,
      n_jobs=n_jobs,
      method='iir',
      iir_params=None
      )

  # 5. Segmentation
  events = get_events(raw_filtered)
  event_dict = {'up/in': 1, 'up/out' : 2, 'inv/in' : 3, 'inv/out' : 4}

  tmin = -0.2
  tmax = 0.5
  baseline = (-0.2, 0)

  epochs = mne.Epochs(
      raw = raw_filtered,
      events=events,
      event_id=event_dict,
      tmin=tmin,
      tmax=tmax,
      baseline=baseline,
      reject_by_annotation=False,
      preload=True,
      reject=None,
      picks=['eeg', 'eog'],
      on_missing = 'warn',
  )

  # 6. Resampling epochs to avoid lost of events
  epochs.resample(sfreq=128)

  # 7. ocular artifact correction with Gratton
  epochs_eog_corrected = ocular_correction_gratton(epochs)

  # 8. Third re-baseline
  epochs_eog_corrected.apply_baseline((-0.2, 0))

  # 9. Artifact correction with autoreject
  ar = AutoReject(random_state=random_state, n_jobs=n_jobs, verbose=0)
  epochs_ar = ar.fit_transform(epochs_eog_corrected, return_log=False)

  return epochs_ar

Set parameters

In [10]:
random_state=42
n_jobs = 10

Perform proprocessing

- Error logs will be stored in: `logger_errors.txt` file, info will be stored in `logger_preprocessing.txt`

- Artifact rejection is performed automatically using [`AutoReject`](https://autoreject.github.io/stable/index.html) python package.

In [ ]:
data_dir = 'D:\studia\magisterka\dane EEG\BADANIE_POLITYCZNE_2022_eeg_bdfy\Dane_EEG' # change to your dir
output_dir = 'D:\studia\magisterka\dane EEG\BADANIE_POLITYCZNE_2022_eeg_bdfy\EEG_preprocessed' # change to your dir

# read all .bdf files from dir path
bdf_files = [f for f in os.listdir(data_dir) if f.endswith(".bdf")]

for file_name in bdf_files:
  bdf_path = os.path.join(data_dir, file_name)

  try:
    pre_processed_epochs = preprocess_file(bdf_path, n_jobs=n_jobs, random_state=random_state)
    output_file_path = os.path.join(output_dir, f"{file_name}-epo.fif")
    pre_processed_epochs.save(output_file_path)


  except Exception as e:
    logger_errors.error(f"{e}")

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    2.6s
[Parallel(n_jobs=10)]: Done  68 out of  68 | elapsed:    4.6s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    0.4s
[Parallel(n_jobs=10)]: Done  68 out of  68 | elapsed:    2.8s finished


Running autoreject on ch_type=eeg




Estimated consensus=0.60 and n_interpolate=4


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    3.0s
[Parallel(n_jobs=10)]: Done  68 out of  68 | elapsed:    4.4s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    0.3s
[Parallel(n_jobs=10)]: Done  68 out of  68 | elapsed:    2.1s finished


Running autoreject on ch_type=eeg




Estimated consensus=0.60 and n_interpolate=32


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    2.6s
[Parallel(n_jobs=10)]: Done  68 out of  68 | elapsed:    4.0s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    0.3s
[Parallel(n_jobs=10)]: Done  68 out of  68 | elapsed:    2.2s finished


...

Running autoreject on ch_type=eeg




Estimated consensus=0.40 and n_interpolate=1


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    2.6s
[Parallel(n_jobs=10)]: Done  68 out of  68 | elapsed:    4.2s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    0.3s
[Parallel(n_jobs=10)]: Done  68 out of  68 | elapsed:    2.2s finished


Running autoreject on ch_type=eeg




Estimated consensus=0.20 and n_interpolate=1


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    2.7s
[Parallel(n_jobs=10)]: Done  68 out of  68 | elapsed:    4.8s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    0.4s
[Parallel(n_jobs=10)]: Done  68 out of  68 | elapsed:    2.8s finished


Running autoreject on ch_type=eeg




Estimated consensus=0.50 and n_interpolate=4
